In [1]:
import os
import numpy as np
import torch
import pydicom
import matplotlib.pyplot as plt
#from tcia_utils import nbia
from monai.bundle import ConfigParser, download

from monai.transforms import (
    LoadImage,
    Rotate90d,
    LoadImaged,
    Orientation,
    Orientationd,
    EnsureChannelFirst,
    EnsureChannelFirstd,
    Compose,
    SaveImaged
)

from rt_utils import RTStructBuilder
import json

from tqdm import tqdm
settings_rel_path = "settings_lung_segment.json"
settings_path = os.path.join(os.path.dirname(__file__), settings_rel_path)

with open(settings_path, 'r') as file:
    settings = json.load(file)


# importances: 0 = normal, 1 = important, 2 = error
def verbose_print(string, importance = 0):
    if importance >= settings["print_treshold"]:
        print(string)
    

verbose_print(settings, importance=0)

input_dir = settings['input_dir']
output_dir = settings['output_dir']


with open(settings['config_path'], 'r') as file:
    config = json.load(file)
inference_path = config["output_dir"]

config = ConfigParser()
config.read_config(settings['config_path'])
preprocessing = config.get_parsed_content("preprocessing")

model = config.get_parsed_content("network")
model.load_state_dict(torch.load(settings["model_path"], map_location=torch.device('cpu')))

inferer = config.get_parsed_content("inferer")
postprocessing = config.get_parsed_content("postprocessing")
verbose_print("Model successfully loaded", importance=0)
#only print if verbose is set to True
#check that input dir exists, if it does not, signal error and exit
#check that output dir exists, if it does not, create it
if not os.path.exists(input_dir):
    verbose_print("ERROR: Input directory does not exist! Terminating program...", importance=2)
    exit(1)
else:
    verbose_print("Input directory found", importance=0)
if not os.path.exists(output_dir):
    verbose_print("Output directory does not exist, creating it...", importance=1)
    os.makedirs(output_dir)
    verbose_print("Output directory created", importance=1)
else:
    verbose_print("Output directory already exists", importance=0)
#list all the files in the input directory
# remove from list all dirs that don't have the prefix settings['prefix']
files = [file for file in os.listdir(input_dir) if file.startswith(settings['prefix']) and not file.endswith(settings['suffix'])]

for file in tqdm(files):
    input_stream = os.path.join(input_dir, file)
    output_stream_folder = os.path.join(output_dir, file)
    
    
    if not os.path.exists(output_stream_folder):
        verbose_print("Output directory does not exist, creating it...", importance=1)
        os.makedirs(output_stream_folder)
        verbose_print("Output directory created", importance=1)
    else:
        verbose_print("Output directory already exists", importance=0)
    
    verbose_print(f"Segmenting {input_stream}", importance=1)

    datalist = [input_stream]
    config['datalist'] = datalist
    dataloader = config.get_parsed_content("dataloader")
    
    #finally we can predict
    X = preprocessing({"image": input_stream})
    with torch.no_grad(): # no backpropagation
        X['pred'] = inferer(X['image'].unsqueeze(0), network = model) # unsqueeze adds a batch dimension
    #this removes the batch dimension in image and pred
    X['pred'] = X['pred'][0]
    X['image'] = X['image'][0]

    X = postprocessing(X)

    X['pred'][X['pred'] < 13] = 0
    X['pred'][X['pred'] > 17] = 0
    image_saver = Compose([SaveImaged(keys = ['pred'], meta_keys = "image_meta_dict", output_dir = output_stream_folder, output_postfix="lung5")])
    image_saver(X)
    verbose_print(f"Five parts segmentation completed", importance=1)
    X['pred'][X['pred'] != 0] = 1
    image_saver = Compose([SaveImaged(keys = ['pred'], meta_keys = "image_meta_dict", output_dir = output_stream_folder, output_postfix="lung")])
    image_saver(X)
    verbose_print(f"Binary segmentation completed", importance=1)
    verbose_print(f"Segmentation completed", importance=1)





    
    
verbose_print("All conversions completed", importance=0)
verbose_print("Program terminated", importance=0)

{'input_dir': 'C:\\Users\\Alessandro\\Desktop\\ML\\MLMLML\\data\\DICOMS', 'output_dir': 'C:\\Users\\Alessandro\\Desktop\\ML\\MLMLML\\data\\niigz', 'model_path': 'C:\\Users\\Alessandro\\Desktop\\ML\\MLMLML\\data\\wholeBody_ct_segmentation\\models\\model_lowres.pt', 'config_path': 'C:\\Users\\Alessandro\\Desktop\\ML\\MLMLML\\data\\wholeBody_ct_segmentation\\configs\\inference.json', 'prefix': 'R01-', 'num_digits': 3, 'separate_seg': True, 'suffix': 'S', 'print_treshold': 0}
Model successfully loaded
Input directory found
Output directory already exists


  0%|          | 0/10 [00:00<?, ?it/s]

Processing file R01-001
Output directory already exists
segmenting C:\Users\Alessandro\Desktop\ML\MLMLML\data\DICOMS\R01-001 into $'C:/Users/Alessandro/Desktop/MLMLML/data/results_whole_body'
2024-02-18 13:21:16,305 INFO image_writer.py:194 - writing: C:\Users\Alessandro\Desktop\ML\MLMLML\data\niigz\R01-001\R01-001\R01-001_lung5.nii.gz
Five parts segmentation completed
2024-02-18 13:21:19,910 INFO image_writer.py:194 - writing: C:\Users\Alessandro\Desktop\ML\MLMLML\data\niigz\R01-001\R01-001\R01-001_lung.nii.gz


 10%|█         | 1/10 [01:05<09:50, 65.66s/it]

Binary segmentation completed
Segmentation completed
Processing file R01-002
Output directory already exists
segmenting C:\Users\Alessandro\Desktop\ML\MLMLML\data\DICOMS\R01-002 into $'C:/Users/Alessandro/Desktop/MLMLML/data/results_whole_body'
